In [2]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 91,
   "metadata": {
    "collapsed": false
   },
   "outputs": [],
   "source": [
    "#加载飞桨和相关类库\n",
    "import paddle\n",
    "from paddle.nn import Linear\n",
    "import paddle.nn.functional as F\n",
    "import numpy as np\n",
    "import random\n",
    "import paddle.vision.transforms as transforms\n",
    "from paddle.metric import Accuracy\n",
    "import matplotlib.pyplot as plt\n",
    "from visualdl import LogWriter"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 92,
   "metadata": {
    "collapsed": false
   },
   "outputs": [],
   "source": [
    "log_writer=LogWriter(\"./data/log/train\") #log记录器\n",
    "transform=transforms.Compose([Normalize(mean=[127.5],\n",
    "                                        std=[127.5],\n",
    "                                        data_format='CHW')])\n",
    "# 设置数据读取器，API自动读取MNIST数据训练集\n",
    "train_dataset = paddle.vision.datasets.MNIST(mode='train',transform=transform)\n",
    "text_dataset = paddle.vision.datasets.MNIST(mode='test',transform=transform)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 93,
   "metadata": {
    "collapsed": false
   },
   "outputs": [],
   "source": [
    "class LeNet(paddle.nn.Layer):\r\n",
    "    def __init__(self):\r\n",
    "        super(LeNet, self).__init__()\r\n",
    "        self.conv1 = paddle.nn.Conv2D(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=2)\r\n",
    "        self.max_pool1 = paddle.nn.MaxPool2D(kernel_size=2,  stride=2)\r\n",
    "        self.conv2 = paddle.nn.Conv2D(in_channels=6, out_channels=16, kernel_size=5, stride=1)\r\n",
    "        self.max_pool2 = paddle.nn.MaxPool2D(kernel_size=2, stride=2)\r\n",
    "        self.linear1 = paddle.nn.Linear(in_features=16*5*5, out_features=120)\r\n",
    "        self.linear2 = paddle.nn.Linear(in_features=120, out_features=84)\r\n",
    "        self.linear3 = paddle.nn.Linear(in_features=84, out_features=10)\r\n",
    "     #前馈网络\r\n",
    "    def forward(self, x):\r\n",
    "        x = self.conv1(x)\r\n",
    "        x = F.relu(x)\r\n",
    "        x = self.max_pool1(x)\r\n",
    "        x = F.relu(x)\r\n",
    "        x = self.conv2(x)\r\n",
    "        x = self.max_pool2(x)\r\n",
    "        x = paddle.flatten(x, start_axis=1,stop_axis=-1)\r\n",
    "        x = self.linear1(x)\r\n",
    "        x = F.relu(x)\r\n",
    "        x = self.linear2(x)\r\n",
    "        x = F.relu(x)\r\n",
    "        x = self.linear3(x)\r\n",
    "        return x\r\n",
    "\r\n",
    "model = paddle.Model(LeNet())   # 封装模型"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 101,
   "metadata": {
    "collapsed": false
   },
   "outputs": [],
   "source": [
    "def train(model,Batch_size=64):\r\n",
    "    # 启动训练模式\r\n",
    "    train_loader = paddle.io.DataLoader(train_dataset, batch_size=Batch_size, shuffle=True)\r\n",
    "    model.train()\r\n",
    "    iterator = 0\r\n",
    "    epochs = 5\r\n",
    "    total_steps = (int(50000//Batch_size)+1)*epochs\r\n",
    "    lr = paddle.optimizer.lr.PolynomialDecay(learning_rate=0.01,decay_steps=total_steps,end_lr=0.001)\r\n",
    "    optim = paddle.optimizer.Adam(learning_rate=0.001, parameters=model.parameters())\r\n",
    "    # 用Adam作为优化函数\r\n",
    "    for epoch in range(epochs):\r\n",
    "        for batch_id, data in enumerate(train_loader()):\r\n",
    "            x_data = data[0]\r\n",
    "            y_data = data[1]\r\n",
    "            predicts = model(x_data)\r\n",
    "            loss = F.cross_entropy(predicts, y_data)\r\n",
    "            # 计算损失\r\n",
    "            acc = paddle.metric.accuracy(predicts, y_data)\r\n",
    "            loss.backward()\r\n",
    "            if batch_id % 200 == 0:\r\n",
    "                print(\"epoch: {}, batch_id: {}, loss is: {}, acc is: {}\".format(epoch, batch_id, loss.numpy(), acc.numpy()))\r\n",
    "                log_writer.add_scalar(tag='acc',step=iterator,value=acc.numpy())\r\n",
    "                log_writer.add_scalar(tag='loss',step=iterator,value=loss.numpy())\r\n",
    "                iterator+=200\r\n",
    "            optim.step()\r\n",
    "            optim.clear_grad()\r\n",
    "        paddle.save(model.state_dict(),'./data/checkpoint/mnist_epoch{}'.format(epoch)+'.pdparams') #保存模型权重\r\n",
    "        paddle.save(optim.state_dict(),'./data/checkpoint/mnist_epoch{}'.format(epoch)+'.pdopt')#保存优化器参数权重\r\n",
    "\r\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 95,
   "metadata": {
    "collapsed": false
   },
   "outputs": [],
   "source": [
    "def test(model):\r\n",
    "    # 加载测试数据集\r\n",
    "    test_loader = paddle.io.DataLoader(test_dataset, places=paddle.CPUPlace(), batch_size=64)\r\n",
    "    model.eval()\r\n",
    "    batch_size = 64\r\n",
    "    for batch_id, data in enumerate(test_loader()):\r\n",
    "        x_data = data[0]\r\n",
    "        y_data = data[1]\r\n",
    "        predicts = model(x_data)\r\n",
    "        # 获取预测结果\r\n",
    "        loss = F.cross_entropy(predicts, y_data)\r\n",
    "        acc = paddle.metric.accuracy(predicts, y_data)\r\n",
    "        if batch_id % 20 == 0:\r\n",
    "            print(\"batch_id: {}, loss is: {}, acc is: {}\".format(batch_id, loss.numpy(), acc.numpy()))\r\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 99,
   "metadata": {
    "collapsed": false
   },
   "outputs": [],
   "source": [
    "#随机抽取100张图片进行测试\r\n",
    "def random_test(model,num=100):\r\n",
    "    # select_id = random.sample(range(1, 10000), 100) #生成一百张测试图片的下标\r\n",
    "    test_loader = paddle.io.DataLoader(test_dataset, places=paddle.CPUPlace(), batch_size=64)\r\n",
    "    for batch_id, data in enumerate(test_loader()):\r\n",
    "        x_data = data[0]\r\n",
    "        label = data[1]\r\n",
    "    predicts = model(x_data)\r\n",
    "    #返回正确率\r\n",
    "    acc = paddle.metric.accuracy(predicts, label)\r\n",
    "    print(\"随机抽样100张图片正确率为：{}\".format(acc.numpy()))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 100,
   "metadata": {
    "collapsed": false
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "epoch: 0, batch_id: 0, loss is: [4.066705], acc is: [0.1875]\n",
      "epoch: 0, batch_id: 200, loss is: [0.24088636], acc is: [0.921875]\n",
      "epoch: 0, batch_id: 400, loss is: [0.03795089], acc is: [0.984375]\n"
     ]
    }
   ],
   "source": [
    "#调用模型\r\n",
    "model = LeNet()\r\n",
    "train(model)\r\n",
    "test(model)\r\n",
    "random_test(model)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "collapsed": false
   },
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "PaddlePaddle 2.1.0 (Python 3.5)",
   "language": "python",
   "name": "py35-paddle1.2.0"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.7.4"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 1
}


NameError: name 'false' is not defined

In [0]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

In [ ]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries

In [ ]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 